# 04. Phân tích & Trực quan hóa Kết quả

**Mục tiêu:** Notebook này sẽ tải file `results/results.csv` (được tạo bởi `03_experiments.ipynb`) và thực hiện phân tích cũng như trực quan hóa chi tiết.

Chúng ta sẽ sử dụng `pandas` để truy vấn dữ liệu và `matplotlib` / `seaborn` để tạo các biểu đồ so sánh, tập trung vào hai chỉ số chính:

1.  **Chất lượng Giải pháp (Solution Quality):** Được đo bằng `gap_percent` (Khoảng cách % so với tối ưu).
2.  **Hiệu suất (Performance):** Được đo bằng `time` (Thời gian chạy, thường ở thang đo log).

## 1. Thiết lập & Tải Dữ liệu

Import các thư viện cần thiết và tải file `results.csv`.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

# Cấu hình hiển thị
%matplotlib inline
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 12

# --- ĐƯỜNG DẪN ---
RESULTS_FILE = '../results/results.csv'
IMAGES_DIR = '../results/charts' # Thư mục để lưu biểu đồ

# Tạo thư mục lưu biểu đồ nếu chưa có
if not os.path.exists(IMAGES_DIR):
    os.makedirs(IMAGES_DIR)
    print(f"Đã tạo thư mục: {IMAGES_DIR}")

# --- Tải Dữ liệu ---
try:
    df = pd.read_csv(RESULTS_FILE)
    print("Tải file 'results.csv' thành công.")
    print(f"Tổng số {len(df)} kết quả được tải.")
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file '{RESULTS_FILE}'.")
    print("Vui lòng chạy notebook '03_experiments.ipynb' trước!")
    df = pd.DataFrame() # Tạo DataFrame rỗng để tránh lỗi

# Hiển thị 5 dòng đầu tiên
if not df.empty:
    display(df.head())

## 2. Phân tích Tổng thể (Cho tất cả các vấn đề)

Hãy xem xét hiệu suất trung bình trên *tất cả* các vấn đề (bao gồm cả các vấn đề nhỏ).

In [ ]:
if not df.empty:
    # Nhóm theo thuật toán và tính trung bình
    df_agg = df.groupby('algorithm')[['gap_percent', 'time']].mean().reset_index()
    
    # Sắp xếp theo Gap (chất lượng)
    df_agg_sorted_gap = df_agg.sort_values(by='gap_percent')
    
    print("--- HIỆU SUẤT TRUNG BÌNH (TẤT CẢ VẤN ĐỀ) ---")
    print(df_agg_sorted_gap.to_string())
else:
    print("DataFrame rỗng, không thể phân tích.")

## 3. Phân tích Vấn đề Lớn (Benchmark)

Đối với các ứng dụng thực tế, chúng ta quan tâm nhiều nhất đến hiệu suất trên các vấn đề lớn (ví dụ: `N > 50`). Hãy lọc dữ liệu để chỉ phân tích các vấn đề này (ví dụ: `berlin52`, `eil76`).

In [ ]:
if not df.empty:
    # Chỉ lọc các vấn đề có N > 50 (loại bỏ 'test_10' và 'burma14')
    # và loại bỏ các thuật toán Exact (chúng không chạy trên các vấn đề này)
    df_benchmark = df[
        (df['dimension'] > 50) & 
        (~df['algorithm'].isin(['Brute Force', 'Held-Karp']))
    ].copy()

    if not df_benchmark.empty:
        print(f"--- Đang phân tích {len(df_benchmark)} kết quả từ các vấn đề lớn (N > 50) ---")
        
        # Nhóm theo thuật toán và tính trung bình
        df_bench_agg = df_benchmark.groupby('algorithm')[['gap_percent', 'time']].mean()
        df_bench_agg = df_bench_agg.sort_values(by='gap_percent')

        display(df_bench_agg)
    else:
        print("Không tìm thấy kết quả nào cho các vấn đề có N > 50.")
else:
    print("DataFrame rỗng, không thể phân tích.")

## 4. Trực quan hóa: Chất lượng Giải pháp (Gap %)

Hãy vẽ biểu đồ cột so sánh `gap_percent` trung bình cho các vấn đề benchmark (N > 50).
**Càng thấp càng tốt.**

In [ ]:
if not df_benchmark.empty:
    # Lấy dữ liệu đã nhóm và sắp xếp
    data_to_plot = df_bench_agg.sort_values(by='gap_percent').reset_index()

    plt.figure(figsize=(14, 8))
    
    # Vẽ biểu đồ cột
    ax = sns.barplot(
        x='gap_percent',
        y='algorithm',
        data=data_to_plot,
        palette='viridis'
    )
    
    ax.set_title('So sánh Chất lượng Giải pháp Trung bình (Vấn đề N > 50)', fontsize=18, fontweight='bold')
    ax.set_xlabel('Khoảng cách % so với Tối ưu (Càng thấp càng tốt)', fontsize=14)
    ax.set_ylabel('Thuật toán', fontsize=14)
    
    # Thêm nhãn giá trị
    for p in ax.patches:
        width = p.get_width()
        ax.text(width + 0.1, # Vị trí x (bên phải cột)
                p.get_y() + p.get_height() / 2, # Vị trí y (ở giữa cột)
                f'{width:.2f}%', # Văn bản
                va='center')
    
    # Lưu biểu đồ
    save_path = os.path.join(IMAGES_DIR, '01_gap_percent_comparison.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Đã lưu biểu đồ vào: {save_path}")
    
    plt.show()
else:
    print("Không có dữ liệu benchmark để vẽ biểu đồ Gap.")

## 5. Trực quan hóa: Thời gian Chạy (Log Scale)

Bây giờ, hãy so sánh thời gian chạy. Vì thời gian chạy có thể chênh lệch rất lớn (từ mili giây đến phút), chúng ta **phải** sử dụng thang đo Logarit (Log Scale) trên trục X.

In [ ]:
if not df_benchmark.empty:
    # Sắp xếp theo thời gian chạy
    data_to_plot_time = df_bench_agg.sort_values(by='time').reset_index()

    plt.figure(figsize=(14, 8))
    
    # Vẽ biểu đồ cột
    ax_time = sns.barplot(
        x='time',
        y='algorithm',
        data=data_to_plot_time,
        palette='plasma'
    )
    
    # !!! ĐẶT TRỤC X SANG THANG ĐO LOG !!!
    ax_time.set_xscale('log')
    
    ax_time.set_title('So sánh Thời gian Chạy Trung bình (Vấn đề N > 50)', fontsize=18, fontweight='bold')
    ax_time.set_xlabel('Thời gian chạy (giây) - (Thang đo Log)', fontsize=14)
    ax_time.set_ylabel('Thuật toán', fontsize=14)
    
    # Thêm nhãn giá trị
    # Nhãn trên thang đo log phức tạp hơn một chút
    for p in ax_time.patches:
        width = p.get_width()
        ax_time.text(width * 1.1, # Vị trí x (hơi bên phải cột)
                     p.get_y() + p.get_height() / 2,
                     f'{width:.3f} s', # Văn bản
                     va='center')
    
    # Lưu biểu đồ
    save_path = os.path.join(IMAGES_DIR, '02_time_comparison_logscale.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Đã lưu biểu đồ vào: {save_path}")
    
    plt.show()
else:
    print("Không có dữ liệu benchmark để vẽ biểu đồ Thời gian.")

## 6. Trực quan hóa: Tương quan Chất lượng vs Thời gian

Một biểu đồ bong bóng (scatterplot) là cách tốt nhất để xem sự đánh đổi (trade-off) giữa Chất lượng (Gap %) và Thời gian (Time).

* **Trục X:** Thời gian (Log)
* **Trục Y:** Chất lượng (Gap %)
* **Mục tiêu:** Góc dưới cùng bên trái (Nhanh & Tốt)

In [ ]:
if not df_benchmark.empty:
    data_to_plot_tradeoff = df_bench_agg.reset_index()

    plt.figure(figsize=(14, 9))
    
    # Vẽ biểu đồ scatterplot với kích thước bong bóng
    ax_tradeoff = sns.scatterplot(
        x='time',
        y='gap_percent',
        hue='algorithm', # Màu sắc cho mỗi thuật toán
        size='gap_percent', # Kích thước bong bóng (tùy chọn)
        sizes=(100, 1000), # Phạm vi kích thước
        legend='full',
        data=data_to_plot_tradeoff,
        palette='tab10'
    )
    
    # Đặt trục X sang thang đo Log
    ax_tradeoff.set_xscale('log')
    
    ax_tradeoff.set_title('Đánh đổi (Trade-off): Chất lượng vs Thời gian (Vấn đề N > 50)', fontsize=18, fontweight='bold')
    ax_tradeoff.set_xlabel('Thời gian chạy (giây) - (Thang đo Log)', fontsize=14)
    ax_tradeoff.set_ylabel('Khoảng cách % so với Tối ưu (Càng thấp càng tốt)', fontsize=14)
    
    # Thêm nhãn cho từng điểm
    for i, row in data_to_plot_tradeoff.iterrows():
        plt.text(row['time'] * 1.1, # Vị trí x
                 row['gap_percent'] + 0.1, # Vị trí y
                 row['algorithm'], # Nhãn
                 fontweight='semibold')
    
    # Di chuyển chú giải (legend) ra ngoài
    ax_tradeoff.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.grid(True, which="both", ls="--")
    
    # Đánh dấu vùng "Lý tưởng"
    # Lấy giá trị min/max
    xmin, xmax = ax_tradeoff.get_xlim()
    ymin, ymax = ax_tradeoff.get_ylim()
    ax_tradeoff.text(xmin * 1.1, ymin * 1.1, "LÝ TƯỞNG\n(Nhanh & Tốt)", 
                     color='green', fontsize=14, fontweight='bold', 
                     ha='left', va='bottom',
                     bbox=dict(facecolor='white', alpha=0.5, edgecolor='green', boxstyle='round,pad=0.5'))
    
    # Lưu biểu đồ
    save_path = os.path.join(IMAGES_DIR, '03_tradeoff_scatterplot.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"Đã lưu biểu đồ vào: {save_path}")
    
    plt.show()
else:
    print("Không có dữ liệu benchmark để vẽ biểu đồ Tương quan.")

## 7. Kết luận Tạm thời

(Dựa trên các biểu đồ,  có thể viết kết luận của mình ở đây. Ví dụ:)

* **Heuristics Đơn giản (NN, NI, Christofides):** Rất nhanh, nhưng chất lượng giải pháp (Gap %) ở mức trung bình.
* **2-Opt:** Cải thiện đáng kể so với NN, và vẫn rất nhanh.
* **Metaheuristics (SA, GA, ACO, TS):** Mất nhiều thời gian nhất, nhưng đạt được chất lượng giải pháp *tốt nhất* (Gap % thấp nhất).
* **Đánh đổi (Trade-off):** `Genetic Algorithm` và `Tabu Search` dường như mang lại sự cân bằng tốt nhất, đạt được Gap % rất thấp trong thời gian chạy hợp lý.